This is a minimal example that obtains the TBR (Tritium Breeding Ratio) for a parametric ball reactor

In [ ]:
import openmc
import paramak
import paramak_neutronics


# makes the 3d geometry from input parameters
my_reactor = paramak.BallReactor(
    inner_bore_radial_thickness=50,
    inboard_tf_leg_radial_thickness=200,
    center_column_shield_radial_thickness=50,
    divertor_radial_thickness=50,
    inner_plasma_gap_radial_thickness=50,
    plasma_radial_thickness=100,
    outer_plasma_gap_radial_thickness=50,
    firstwall_radial_thickness=1,
    blanket_radial_thickness=100,
    blanket_rear_wall_radial_thickness=10,
    elongation=2,
    triangularity=0.55,
    number_of_tf_coils=16,
    rotation_angle=359.9,  # when using Cubit method this can be set to 360
)

# method is set to Cubit or Cubit by default to avoid overlaps in the geometry
# pymoab is used as it is open source and can be tested in the CI
# if you have Cubit then this line can be deleted
my_reactor.method='pymoab'

source = openmc.Source()
# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((my_reactor.major_radius, 0, 0))
# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()
# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

# makes the neutronics model from the geometry and material allocations
neutronics_model = paramak_neutronics.NeutronicsModel(
    h5m_filename=my_reactor.export_h5m(),
    source=source,
    materials={
        'inboard_tf_coils_mat': 'eurofer',
        'center_column_shield_mat': 'eurofer',
        'divertor_mat': 'eurofer',
        'firstwall_mat': 'eurofer',
        'blanket_rear_wall_mat': 'eurofer',
        'blanket_mat': 'Li4SiO4'},
    cell_tallies=['TBR', 'heating'],
    simulation_batches=2,
    simulation_particles_per_batch=10,  # this will need increasing to obtain accurate results
)

# simulate the neutronics model
neutronics_model.simulate()
print(neutronics_model.results)
